## 1. Notebook for predicting habit of lot of images
* loop through images - predict most likely habit + all habit possibilities
* save to csv

In [12]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import register_keras_serializable
import csv
from glob import glob
import os
import pandas as pd

In [40]:
## where is the data
# Jaffeux data -> is the training data used for the model
Jaffeux_data = '/gws/nopw/j04/dcmex/users/ezriab/cp-of-Jaffeux-22/DS_training_data/' 
j_category = 'class_05_CG/' #class_01_Pri  class_02_Co  class_03_CC  class_04_ABC  class_05_CG  class_06_HC  class_07_Di  class_08_FA  class_09_WD
dcmex_2ds = '/gws/nopw/j04/dcmex/users/ezriab/2dprocessed/flight_220730153000/'

file_path = dcmex_2ds # path to all files

# get all files in specif folder
file_list = glob(file_path+'*.png') # whole string of path + filenames in location
file_names = [os.path.basename(file_path) for file_path in file_list] # just file names


## where would you like predictions saved?
save_loc = '/gws/nopw/j04/dcmex/users/ezriab/2dprocessed/flight_220730153000/'
save_name = 'habit_predictions.csv'

## quite important
categories = ['CA', 'Co',  'CC', 'CBC', 'CG', 'HPC', 'Dif', 'FA', 'WD']

In [41]:
## open model 
model_path = '/gws/nopw/j04/dcmex/users/ezriab/cp-of-Jaffeux-22/novmodel.h5py' # to h5 folder

# Define custom layer with serialization support
# (model would not open without this)
@register_keras_serializable()
class RandomFlip(tf.keras.layers.Layer):
    def __init__(self, mode='horizontal_and_vertical', seed=None, **kwargs):
        super(RandomFlip, self).__init__(**kwargs)
        self.mode = mode
        self.seed = seed

    def call(self, inputs, training=None):
        if training:
            if self.mode == 'horizontal_and_vertical':
                return tf.image.random_flip_left_right(tf.image.random_flip_up_down(inputs), seed=self.seed)
            elif self.mode == 'horizontal':
                return tf.image.random_flip_left_right(inputs, seed=self.seed)
            elif self.mode == 'vertical':
                return tf.image.random_flip_up_down(inputs, seed=self.seed)
        return inputs

    def get_config(self):
        config = super(RandomFlip, self).get_config()
        config.update({'mode': self.mode, 'seed': self.seed})
        return config

# Load the model with custom objects
custom_objects = {'RandomFlip': RandomFlip}
model = load_model(model_path, custom_objects=custom_objects)

## may also want to see summary of model -> check its doing correct things
# model.summary()

In [42]:
# list to store results
results =[]

for i in range(len(file_names)):
    
    img = image.load_img(file_path+file_names[i], target_size=(200, 200), color_mode='grayscale')
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    
    # Predict using the loaded model
    predictions = model.predict(img_array)[0]
    # Find the index of the category with the highest probability
    predicted_index = np.argmax(predictions)
    
    # Get the corresponding category
    predicted_category = categories[predicted_index]

    # Prepare the data to be saved
    data = {'Name': file_names[i][:-4], 'Category': predicted_category}
    for i, category in enumerate(categories):
        data[category] = predictions[i]
    
    # Append the data to results list
    results.append(data)

df = pd.DataFrame(results, columns=columns_names)
# Save the DataFrame to a CSV file
df.to_csv(save_loc+save_name, index=False)

1/1 [==============================] - 0s 56ms/step
